<a href="https://colab.research.google.com/github/michunie99/RL-course-huggin-face/blob/main/Car_Racing_PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 2s (518 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 124013 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpack

In [2]:
!pip install -r https://huggingface.co/spaces/ThomasSimonini/temp-space-requirements/raw/main/requirements/requirements-unit1.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 30.8 MB/s 
     |████████████████████████████████| 1.3 MB 49.8 MB/s 
     |████████████████████████████████| 425 kB 73.3 MB/s 
     |████████████████████████████████| 1.0 MB 15.9 MB/s 
     |████████████████████████████████| 182 kB 75.5 MB/s 
     |████████████████████████████████| 1.5 MB 60.8 MB/s 
     |████████████████████████████████| 237 kB 75.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51 kB 1.2 MB/s 
  Created wheel for box2d-kengz: filename=Box2D_kengz-2.3.3-cp38-cp38-linux_x86_64.whl size=2043236 sha256=080dbbb7a9b145ede175dc5b05381578847438c5c0b056cccd3b98a8cb7a5351
  Stored in directory: /root/.cache/pip/wheels/87/3a/ed/260cc09ed176c5b06aed67364b2

In [3]:
!sudo apt-get update
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,073 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,567 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InR

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gym

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

In [3]:
# Cell for video saving 
# Taken form: https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_12_01_ai_gym.ipynb#scrollTo=T9RpF49oOsZj
from gym.wrappers import Monitor 
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")


def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

In [4]:
import gym
from gym.spaces import Discrete, Box
import numpy as np

class DiscreteActions(gym.ActionWrapper):
    def __init__(self, env, disc_to_cont):
        super().__init__(env)
        self.disc_to_cont = disc_to_cont
        self.action_space = Discrete(len(disc_to_cont))
    
    def action(self, act):
        return self.disc_to_cont[act]

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, max_reward):
        super().__init__(env)
        self.min_reward = -0.1
        self.max_reward = max_reward
        self.reward_range = (-0.1, max_reward)
    
    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)

In [28]:
from gym.wrappers import GrayScaleObservation, FrameStack, TransformObservation

def create_env():
  # Create a wraper for the enviroment
  env = gym.make("CarRacing-v0")
  # Observations conversionn to gray scale

  # Step 1 - crop observation (84,96) removes the lower bar
  wrapped_env = TransformObservation(env, lambda obs: obs[:84,:,:])

  # Adjust shape 
  wrapped_env.observation_space = Box(0, 255, (84,96,3))

  #TODO - normalizacja ???

  # Step 2 - Convert observation to gray scale
  wrapped_env = GrayScaleObservation(wrapped_env, keep_dim=True)

  # Step 3 - stack last 4 frames
  # wrapped_env = FrameStack(wrapped_env, num_stack=4)

  # Action convert to a descrete action space
  # Action 1 - [-1, 0, 0] full turn left
  # Action 2 - [1, 0, 0] full turn right
  # Action 3 - [0, 1, 0] full acceleration
  # Action 4 - [0, 0, 0.8] full breaking
  # Action 5 - [0, 0, 0] nothing

  # My custorm actions array
  actions = [np.array([-1, 0, 0]), np.array([1, 0, 0]), np.array([0, 0.8, 0]),
            np.array([0, 0, 0.8]), np.array([0, 0, 0])]

  wrapped_env = DiscreteActions(wrapped_env, actions)

  # Clip the reward
  wrapped_env = ClipReward(wrapped_env, 1)

  return wrapped_env

In [6]:
wrapped_env = create_env()
print(f"Number of observations {wrapped_env.observation_space.shape}")
print(f"Number of actions {wrapped_env.action_space}")

Number of observations (84, 96, 1)
Number of actions Discrete(5)


In [103]:
# Random initial agent 
env = wrap_env(wrapped_env)

observation = env.reset()

for _ in range(200):
  # Take a random action
  action = env.action_space.sample()

  # Do this action in the environment and get
  # next_state, reward, done and info
  observation, reward, done, info = env.step(action)
  
  # If the game is done (in our case we land, crashed or timeout)
  if done:
      # Reset the environment
      print("Environment is reset")
      observation = env.reset()

env.close()
show_video()

Track generation: 1194..1497 -> 303-tiles track

In [15]:
# Create vectorized enviroments
env = make_vec_env(create_env, n_envs=6)

In [22]:
# Create PPO model form Stable-Baseline3
model = PPO(
    policy='CnnPolicy', #CNN policy the observation is an image
    env=env,
    n_epochs= 10,
    n_steps = 1024,
    batch_size = 32
)

In [23]:
# Train model
model_name = "ppo-car-racing-v0-modified"
model.learn(
    total_timesteps= 500000, 
    progress_bar = True)
model.save(model_name)

Output()

Track generation: 1207..1513 -> 306-tiles track


Track generation: 1210..1517 -> 307-tiles track

Track generation: 1086..1363 -> 277-tiles track

Track generation: 1181..1480 -> 299-tiles track

...

Track generation: 1188..1489 -> 301-tiles track

Track generation: 1135..1423 -> 288-tiles track

Track generation: 1184..1484 -> 300-tiles track

Track generation: 1174..1471 -> 297-tiles track

Track generation: 1206..1511 -> 305-tiles track

In [24]:
# TODO: Evaluate the agent
# model = PPO.load("ppo-lunal-lander-v2")
model = PPO.load(model_name)
# Create a new environment for evaluation
eval_env = create_env()

# Evaluate the model with 10 evaluation episodes and deterministic=True
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)

# Print the results
print(f"Mean reward is: {mean_reward}, std reward: {std_reward}")

# 1-st try (5 epoch): Mean reward is: -40.27, std reward: 22.50
# 2-nd try (10 epoch): Mean reward is: -75.69, std reward: 5.14 (Change acceleration smaller)
# 3-rd try (10 epoch): Mean reward is: -75.25, std reward: 4.97

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Track generation: 1096..1382 -> 286-tiles track
Track generation: 1264..1584 -> 320-tiles track
Track generation: 1152..1444 -> 292-tiles track
Track generation: 1104..1384 -> 280-tiles track
Track generation: 1177..1475 -> 298-tiles track
Track generation: 1138..1433 -> 295-tiles track
Track generation: 1043..1315 -> 272-tiles track
Track generation: 1269..1590 -> 321-tiles track
Track generation: 1212..1519 -> 307-tiles track
Track generation: 1371..1718 -> 347-tiles track
Track generation: 1194..1503 -> 309-tiles track
Mean reward is: -75.2500014565885, std reward: 4.974384390760361


In [25]:
# Test the model in the enviroment
# Random initial agent 
env = wrap_env(eval_env)

observation = env.reset()

for _ in range(50000):
  # Take a random action

  action, _states = model.predict(observation)

  # Do this action in the environment and get
  # next_state, reward, done and info
  observation, reward, done, info = env.step(action)
  
  # If the game is done (in our case we land, crashed or timeout)
  if done:
      # Reset the environment
      print("Environment is reset")
      observation = env.reset()

env.close()
show_video()

Track generation: 963..1212 -> 249-tiles track
Environment is reset
Track generation: 1103..1387 -> 284-tiles track
Environment is reset
Track generation: 1065..1335 -> 270-tiles track
Environment is reset
Track generation: 1272..1594 -> 322-tiles track
Environment is reset
Track generation: 1196..1499 -> 303-tiles track
Environment is reset
Track generation: 1052..1319 -> 267-tiles track
Environment is reset
Track generation: 1184..1484 -> 300-tiles track
Environment is reset
Track generation: 1247..1563 -> 316-tiles track
Environment is reset
Track generation: 1213..1524 -> 311-tiles track
Environment is reset
Track generation: 1118..1409 -> 291-tiles track
Environment is reset
Track generation: 1307..1638 -> 331-tiles track
Environment is reset
Track generation: 1123..1408 -> 285-tiles track
Environment is reset
Track generation: 1196..1499 -> 303-tiles track
Environment is reset
Track generation: 986..1243 -> 257-tiles track
Environment is reset
Track generation: 952..1194 -> 242-t

# Save the model to **Huggin Face**

In [ ]:
from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

In [ ]:
notebook_login()
!git config --global credential.helper store

In [ ]:
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

## TODO: Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
repo_id = "Michunie/ppo-CarRacing-v0"

# TODO: Define the name of the environment
env_id = "CarRacing-v0"

# Create the evaluation env
eval_env = DummyVecEnv([lambda: gym.make(env_id)])


# TODO: Define the model architecture we used
model_architecture = "PPO"

## TODO: Define the commit message
commit_message = "Attempt at CarRacing env"

# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model 
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

# Note: if after running the package_to_hub function and it gives an issue of rebasing, please run the following code
# cd <path_to_repo> && git add . && git commit -m "Add message" && git pull 
# And don't forget to do a "git push" at the end to push the change to the hub.